In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
#Get  the starting seed state
Seed=np.random.get_state()
%store -r 
#apply a certain previously stored seed to the algorithm
np.random.set_state(seed_4stages6_t_7_76)

def wave_effect(alpha,delta):
    '''
    Function that represents effect of each wave plate in the SOP
    '''
    
    wave_plate_matrix=[[np.cos(alpha)**2+(np.sin(alpha)**2)*np.cos(delta), np.cos(alpha)*np.sin(alpha)*(1-np.cos(delta)),np.sin(alpha)*np.sin(delta)],
                  [np.cos(alpha)*np.sin(alpha)*(1-np.cos(delta)),np.cos(delta)*np.cos(alpha)**2+np.sin(alpha)**2,-np.cos(alpha)*np.sin(delta)],
                  [-np.sin(alpha)*np.sin(delta),np.cos(alpha)*np.sin(delta),np.cos(delta)]]

    return wave_plate_matrix 

def V_A_value(alpha,delta,V_o,V_pi,V_A_bias):
    
    ''' 
    V A calculation
    '''
    
    V_A=2*V_o*delta*np.sin(alpha)-V_pi*delta*np.cos(alpha)+V_A_bias

    return V_A
 
def V_C_value(alpha,delta,V_o,V_pi,V_C_bias):
    
    ''' 
    V C calculation
    '''
    V_C=2*V_o*delta*np.sin(alpha)+V_pi*delta*np.cos(alpha)+V_C_bias

    return V_C

def f_lab(alpha_array,delta_array,V_Ain_matrix,V_Cin_matrix,start_array,goal_array,first_weight,sec_weight,third_weight,extra_sum,n_stages):
    ''' 
    alpha_array - array with the values of alpha for the various stages of a PSO particle
    delta_array - array with the values of delta for the various stages of a PSO particle
    V_Ain_matrix - matrix of values of VA for the previous polarization state for each stage
    V_Cin_matrix - matrix of values of VC for the previous polarization state for each stage
    start_array - Input State of Polarization as a Stokes Vector
    goal_array - Goal State of Polarization as a Stokes Vector
    first_weight,sec_weight,third_weight - weights of the cost function
    extra_sum - 1 or 0 , 1 if we are comparing to more than one previous calculated SOP
    n_stages - number of stages of the EPC used
    '''

    
    V_pi_array=np.array([56.3,56,55.9,56.1,56.1,56,56.1,56])
    V_o_array=np.array([27.2,27,27,26.9,26.8,27.2,26.8,27.2])

    #V_A,bias e V_C,bias não têm efeito no processo de otimização 
    V_A_bias_array=np.array([-10.7,-9.3,-8.5,-10.9,-7.4,-7.6,-7.4,-7.6])
    V_C_bias_array=np.array([8.4,9.6,9.4,11.1,11.6,10.9,11.6,10.9])

    first_sum=0
    second_sum=0
    third_sum=0
    
    delta_array_rad=delta_array*np.pi*2
    
    M=wave_effect(alpha_array[n_stages-1],delta_array_rad[n_stages-1])
    i=n_stages-2
    while(i>=0): 
        M=np.dot(M,wave_effect(alpha_array[i],delta_array_rad[i]))
        i-=1
    
    start_array=np.reshape(start_array,(3,1))
    
    calc_array=np.dot(M,start_array)
    
    calc_array=np.reshape(calc_array,(1,3))
    calc_array=calc_array[0]
    
    for i in range(0,3):
        first_sum+=abs((calc_array[i]-goal_array[i]))*first_weight
        
    
    if(extra_sum):   
        

        for j in range(0,len(V_Ain_matrix)):

            for i in range(0,n_stages):
                
                second_sum+=(abs(V_A_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_A_bias_array[i])-V_Ain_matrix[j,i])/(sec_weight*len(V_Ain_matrix)))**2


        for j in range(0,len(V_Cin_matrix)):
            for i in range(0,n_stages):
                third_sum+=(abs(V_C_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_C_bias_array[i])-V_Cin_matrix[j,i])/(third_weight*len(V_Cin_matrix)))**2
                
    else:
        for i in range(0,n_stages):
            temp_calc=(abs((V_A_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_A_bias_array[i])-V_Ain_matrix[i]))/sec_weight)**2
            second_sum+=temp_calc

    
        for i in range(0,n_stages):
            temp_calc=(abs((V_C_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_C_bias_array[i])-V_Cin_matrix[i]))/third_weight)**2
            third_sum+=temp_calc


    z=first_sum+second_sum+third_sum

    return z


In [2]:
def index_array(lower_Q1_delta,lower_Q1_alpha,V_A_std,alpha_increment,delta_increment):
    
    ''' 
    This function is called by the function interval_selection and it 
    serves to find the index breraking points
    '''
    
    index_alpha_array=[]
    for item in range(0,V_A_std[1,:].size):
        
        if(V_A_std[1,item] <= lower_Q1_alpha):
            index_alpha_array.append(item)
        
    index_delta_array=[]

    for item in range(0,V_A_std[0,:].size):
        
        if(V_A_std[0,item] <= lower_Q1_delta):
            index_delta_array.append(item)
            

    i=0   
    sequence_delta_array=[]
    while(i<(len(index_delta_array)-1)):

        if((index_delta_array[i]-index_delta_array[i+1])!=-1):
            
            sequence_delta_array.append(i)
            
        i+=1


    i=0   
    sequence_alpha_array=[]
    while(i<(len(index_alpha_array)-1)):

        if((index_alpha_array[i]-index_alpha_array[i+1])!=-1):
            
            sequence_alpha_array.append(i)
            
        i+=1
        

    
    start_delta_array=[]
    end_delta_array=[]
    if(sequence_delta_array==[]):
        start_delta_array.append(delta_increment*index_delta_array[0])
        end_delta_array.append(delta_increment*index_delta_array[-1])
    else:
        start=1
        
        for item in range(0,len(index_delta_array)):
            if(start==1):
                start_delta_array.append(delta_increment*index_delta_array[item])
                start=0
            if(item in sequence_delta_array):
                end_delta_array.append(delta_increment*index_delta_array[item])
                start=1

        end_delta_array.append(delta_increment*index_delta_array[-1])
        
    start_alpha_array=[]
    end_alpha_array=[]
    if(sequence_alpha_array==[]):
        start_alpha_array.append(alpha_increment*index_alpha_array[0])
        end_alpha_array.append(alpha_increment*index_alpha_array[-1])
    else:

        start=1
        
        for item in range(0,len(index_alpha_array)):
            if(start==1):
                start_alpha_array.append(alpha_increment*index_alpha_array[item])
                start=0
            if(item in sequence_alpha_array):
                end_alpha_array.append(alpha_increment*index_alpha_array[item])
                start=1

        end_alpha_array.append(alpha_increment*index_alpha_array[-1])

    return(index_delta_array,index_alpha_array,start_delta_array,end_delta_array,start_alpha_array,end_alpha_array)


In [3]:
def interval_selection(V_pi,V_0,V_a_bias,V_c_bias):
    
    ''' 
    Calculates the starting particle space
    '''
    
    #number of alpha and delta divitions divitons in their range
    size=50

    ###
    alpha_array=np.zeros(size)
    delta_array=np.zeros(size)

    alpha_increment=(2*np.pi)/(alpha_array.size-1)
    delta_increment=1/(delta_array.size-1)


    alpha=0
    i=0
    while i<alpha_array.size:
        alpha_array[i]=alpha
        alpha+=alpha_increment
        i+=1


    delta=0
    i=0
    while i<delta_array.size:
        delta_array[i]=delta
        delta+=delta_increment
        i+=1

    V_a=np.zeros((delta_array.size,alpha_array.size))

    i=0
    while i<delta_array.size:
        j=0
        while j<alpha_array.size:
            V_a[i,j]=2*V_0*delta_array[i]*np.sin(alpha_array[j])-V_pi*delta_array[i]*np.cos(alpha_array[j])+V_a_bias
            j+=1
        i+=1

    V_c=np.zeros((delta_array.size,alpha_array.size))
    i=0
    while i<delta_array.size:
        j=0
        while j<alpha_array.size:
            V_c[i,j]=2*V_0*delta_array[i]*np.sin(alpha_array[j])+V_pi*delta_array[i]*np.cos(alpha_array[j])+V_c_bias
            j+=1
        i+=1
    
    V_C_std=np.zeros((2,size))
    V_A_std=np.zeros((2,size))

    ''' 
    first row has the  standard  deviation of the delta rows
    second row has the standard deviation of the alpha columns
    '''

    j=0
    while(j<2):
        i=0
        if(j==0):
            while(i<size):
                V_C_std[j,i]=np.std(V_c[i,:])
                V_A_std[j,i]=np.std(V_a[i,:])
                i+=1
        else:
            while(i<size):
                V_C_std[j,i]=np.std(V_c[:,i])
                V_A_std[j,i]=np.std(V_a[:,i])
                i+=1
        j+=1


    lower_Q1_delta=np.quantile(V_A_std[0,:],0.15)
    lower_Q1_alpha=np.quantile(V_A_std[1,:],0.15)

    index_delta_array_A,index_alpha_array_A,start_delta_array_A,end_delta_array_A,start_alpha_array_A,end_alpha_array_A=index_array(lower_Q1_delta,lower_Q1_alpha,V_A_std,alpha_increment,delta_increment)
    index_delta_array_C,index_alpha_array_C,start_delta_array_C,end_delta_array_C,start_alpha_array_C,end_alpha_array_C=index_array(lower_Q1_delta,lower_Q1_alpha,V_C_std,alpha_increment,delta_increment)

    V_C_delta_mean=V_c[index_delta_array_C,:]
    V_A_delta_mean=V_a[index_delta_array_A,:]
    V_C_alpha_mean=V_c[:,index_alpha_array_C]
    V_A_alpha_mean=V_a[:,index_alpha_array_A]

    V_A_mean=(np.sum(V_A_delta_mean)+np.sum(V_A_alpha_mean))/(np.size(V_A_delta_mean[0])*np.size(V_A_delta_mean[:,0])+(np.size(V_A_alpha_mean[0])*np.size(V_A_alpha_mean[:,0])))

    V_C_mean=(np.sum(V_C_delta_mean)+np.sum(V_C_alpha_mean))/(np.size(V_C_delta_mean[0])*np.size(V_C_delta_mean[:,0])+(np.size(V_C_alpha_mean[0])*np.size(V_C_alpha_mean[:,0])))
    

    return(start_delta_array_A,start_delta_array_C,end_delta_array_A,end_delta_array_C,start_alpha_array_A,start_alpha_array_C,end_alpha_array_A,end_alpha_array_C,V_A_mean,V_C_mean)

In [4]:
def particle_distribution(n_particles,start_delta_array,end_delta_array,start_alpha_array,end_alpha_array):
    
    ''' Used to calculate random distribuitions used for the particles'''
    
    val_delta=1
    val_alpha=1
    j=0
    counter=0
    j_delta=n_particles/len(start_delta_array)

    while(j<n_particles):
        
        if(val_delta==1):
            X_delta=np.random.rand(1, int(j_delta))*(end_delta_array[0]-start_delta_array[0])+start_delta_array[0]
            X_delta=X_delta[0]
            val_delta=0
    
        else:
            X_delta_add=np.random.rand(1, int(j_delta))*(end_delta_array[counter]-start_delta_array[counter])+start_delta_array[counter]
            X_delta_add=X_delta_add[0]                                                                                                              
            X_delta = np.concatenate((X_delta, X_delta_add))

        counter+=1
        j+=j_delta


    
    l=0
    counter=0
    l_delta=n_particles/len(start_alpha_array)
    while(l<n_particles):

        if(val_alpha==1):
            X_alpha=np.random.rand(1, int(l_delta))*(end_alpha_array[0]-start_alpha_array[0])+end_alpha_array[0]
            X_alpha=X_alpha[0]
            val_alpha=0
        else:
            X_alpha_add=np.random.rand(1, int(l_delta))*(end_alpha_array[counter]-start_alpha_array[counter])+start_alpha_array[counter]
            X_alpha_add=X_alpha_add[0]
            X_alpha = np.concatenate((X_alpha, X_alpha_add))

            
        counter+=1
        l+=l_delta
    
    return(X_delta,X_alpha)

In [5]:
def PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,start_array,iteration_max,first_weight,sec_weight,third_weight,extra_sum,n_stages):
    
    ''' Particle swarm optimization implementation '''
    #since the particles havent started exploring their best position is their current one 
    pbest=X
    #we apply the coordenates of each particle to the function
    pbest_obj=np.zeros(n_particles)

    for i in range(0,n_particles):
        pbest_obj[i] = f_lab(X[n_stages:(n_stages*2)][:,i], X[0:n_stages][:,i],V_A_in_array,V_C_in_array,start_array,goal_array,first_weight,sec_weight,third_weight,extra_sum,n_stages)
    #pick the coordinates of the particle with the lowest values
    gbest = pbest[:, pbest_obj.argmin()]
    #place the value 
    gbest_obj = pbest_obj.min()
    
    iterations=0

    
    while(iterations<iteration_max):
        #algorithm constants
        c1 = c2 = 0.2
        w = 0.6
        #random factor
        r = np.random.rand(2)
        #particle actualization
        #we subtract both both matrixeses --- we subtract the column of best values to all the entire column of X 
        V = w * V + c1*r[0]*(pbest - X) + c2*r[1]*(gbest.reshape(-1,1)-X)
        X = X + V

        
        obj=np.zeros(n_particles)
        for i in range(0,n_particles):
            obj[i] = f_lab(X[n_stages:(n_stages*2)][:,i], X[0:n_stages][:,i],V_A_in_array,V_C_in_array,start_array,goal_array,first_weight,sec_weight,third_weight,extra_sum,n_stages)
            
        #we subtract the coordenate vector with the coordenates of the lowest value obtained up until now 
        pbest[:, (pbest_obj >= obj)] = X[:, (pbest_obj >= obj)]
        #update the best value obtained until now
        pbest_obj = np.array([pbest_obj, obj]).min(axis=0)
        gbest = pbest[:, pbest_obj.argmin()]
        gbest_obj = pbest_obj.min()
        

        iterations+=1

    return(gbest_obj,gbest)

In [6]:
def output_calc(gbest,input_pol,n_stages):
    
    ''' Calculation of the output polarization '''
    
    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]
    delta_array_rad=delta_array*np.pi*2

    input_pol=np.reshape(input_pol,(3,1))
    
    M=wave_effect(alpha_array[n_stages-1],delta_array_rad[n_stages-1])
    i=n_stages-2
    while(i>=0): 
        M=np.dot(M,wave_effect(alpha_array[i],delta_array_rad[i]))
        i-=1
    
    output_pol=np.dot(M,input_pol)
    
    output_pol=np.reshape(output_pol,(1,3))
    output_pol=output_pol[0]
    
    return(output_pol)
    

In [7]:
def rise_time(delta_array,alpha_array,V_Ain_array,V_Cin_array,n_stages):
    ''' Voltage intervals calculation '''
    V_A_measurment=np.zeros(n_stages)
    V_C_measurment=np.zeros(n_stages)

    V_pi_array=np.array([56.3,56,55.9,56.1,56.1,56,56.1,56])
    V_o_array=np.array([27.2,27,27,26.9,26.8,27.2,26.8,27.2])


    #V_A,bias e V_C,bias have no effect on the optimization process
    V_A_bias_array=np.array([-10.7,-9.3,-8.5,-10.9,-7.4,-7.6,-7.4,-7.6])
    V_C_bias_array=np.array([8.4,9.6,9.4,11.1,11.6,10.9,11.6,10.9])


    i=0
    while(i<n_stages):
        V_A_measurment[i]=V_A_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_A_bias_array[i])-V_Ain_array[i]
        i+=1
    i=0
    while(i<n_stages):
        V_C_measurment[i]=V_C_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_C_bias_array[i])-V_Cin_array[i]
        i+=1
    
    V_A_measurment_abs=abs(V_A_measurment)
    V_C_measurment_abs=abs(V_C_measurment)


    V_A_measurment_val=np.zeros(n_stages)
    V_C_measurment_val=np.zeros(n_stages)
    i=0
    while(i<n_stages):
        V_A_measurment_val[i]=V_A_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_A_bias_array[i])
        i+=1
    i=0
    while(i<n_stages):
        V_C_measurment_val[i]=V_C_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_C_bias_array[i])
        i+=1
    
    
    return(V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs)

In [8]:
def particle_space(r_start_delta_array_A,r_start_delta_array_C,r_end_delta_array_A,r_end_delta_array_C,r_start_alpha_array_A,r_start_alpha_array_C,r_end_alpha_array_A,r_end_alpha_array_C,n_stages,n_particles):
    
    ''' gives the matrix of new coordinatinates for the particles  '''
    
    X_delta=np.array([])
    X_alpha=np.array([])
    i=0
    
    while(i<n_stages):
        
        start_delta_array_A=r_start_delta_array_A[i]
        start_delta_array_C=r_start_delta_array_C[i]
        end_delta_array_A=r_end_delta_array_A[i]
        end_delta_array_C=r_end_delta_array_C[i]
        start_alpha_array_A=r_start_alpha_array_A[i]
        start_alpha_array_C=r_start_alpha_array_C[i]
        end_alpha_array_A=r_end_alpha_array_A[i]
        end_alpha_array_C=r_end_alpha_array_C[i]
        
        X_delta_add,X_alpha_add=particle_distribution(n_particles/2,start_delta_array_A,end_delta_array_A,start_alpha_array_A,end_alpha_array_A)
        
        X_delta=np.concatenate((X_delta,X_delta_add))
        X_alpha=np.concatenate((X_alpha,X_alpha_add))
        
        X_delta_add,X_alpha_add=particle_distribution(n_particles/2,start_delta_array_C,end_delta_array_C,start_alpha_array_C,end_alpha_array_C)
        
        X_delta=np.concatenate((X_delta,X_delta_add))
        X_alpha=np.concatenate((X_alpha,X_alpha_add))
        
        i+=1
        
    X_delta=np.reshape(X_delta,(n_stages,n_particles))
    X_alpha=np.reshape(X_alpha,(n_stages,n_particles))
    
    return(X_delta,X_alpha)

## Starting H state

In [9]:
start = time.time()
V_pi_array=np.array([56.3,56,55.9,56.1,56.1,56,56.1,56])
V_o_array=np.array([27.2,27,27,26.9,26.8,27.2,26.8,27.2])
V_A_bias_array=np.array([-10.7,-9.3,-8.5,-10.9,-7.4,-7.6,-7.4,-7.6])
V_C_bias_array=np.array([8.4,9.6,9.4,11.1,11.6,10.9,11.6,10.9])

''' important parameters '''
goal_array=np.array([1,0,0])
input_pol=np.array([ 0.55916189 , 0.12389344 , -0.91504942]) #<--lab input SOP
#input_pol=np.array([1,0,0])
reference_point=np.sqrt(input_pol[0]**2+input_pol[1]**2+input_pol[2]**2)

# np.random.seed(seed=0)
n_particles = 80
n_stages=6
''' -------------------------- '''

r_start_delta_array_A=[]
r_start_delta_array_C=[]
r_end_delta_array_A=[]
r_end_delta_array_C=[]
r_start_alpha_array_A=[]
r_start_alpha_array_C=[]
r_end_alpha_array_A=[]
r_end_alpha_array_C=[]

output_pol=np.zeros(3)
cond=1
while(cond==1):
    
    V_A_in_array=np.zeros(n_stages)
    V_C_in_array=np.zeros(n_stages)
    X_delta=np.array([])
    X_alpha=np.array([])
    i=0
    
    while(i<n_stages):
        
        start_delta_array_A,start_delta_array_C,end_delta_array_A,end_delta_array_C,start_alpha_array_A,start_alpha_array_C,end_alpha_array_A,end_alpha_array_C,V_A_in,V_C_in=interval_selection(V_pi_array[i],V_o_array[i],V_A_bias_array[i],V_C_bias_array[i])
        
        r_start_delta_array_A.append(start_delta_array_A)
        r_start_delta_array_C.append(start_delta_array_C)
        r_end_delta_array_A.append(end_delta_array_A)
        r_end_delta_array_C.append(end_delta_array_C)
        r_start_alpha_array_A.append(start_alpha_array_A)
        r_start_alpha_array_C.append(start_alpha_array_C)
        r_end_alpha_array_A.append(end_alpha_array_A)
        r_end_alpha_array_C.append(end_alpha_array_C)

        
        V_A_in_array[i]=V_A_in
        V_C_in_array[i]=V_C_in
        
        X_delta_add,X_alpha_add=particle_distribution(n_particles/2,start_delta_array_A,end_delta_array_A,start_alpha_array_A,end_alpha_array_A)
        
        X_delta=np.concatenate((X_delta,X_delta_add))
        X_alpha=np.concatenate((X_alpha,X_alpha_add))
        
        X_delta_add,X_alpha_add=particle_distribution(n_particles/2,start_delta_array_C,end_delta_array_C,start_alpha_array_C,end_alpha_array_C)
        
        X_delta=np.concatenate((X_delta,X_delta_add))
        X_alpha=np.concatenate((X_alpha,X_alpha_add))
        
        i+=1
        
    ''' 
    it is possible for errors to occur at reshape if the (number of particles)/len(...) is not a whole number,  if the number of particles is divisible by 4 there should be
    no problems.
    '''
    
    
    X_delta=np.reshape(X_delta,(n_stages,n_particles))
    X_alpha=np.reshape(X_alpha,(n_stages,n_particles))

    X = np.concatenate((X_delta, X_alpha))
    V = np.random.randn(n_stages*2, n_particles) * 0.1 #normal distribution

    gbest_obj,gbest=PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,input_pol,250,2,10,10,0,n_stages)

    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]

    
    output_pol=output_calc(gbest,input_pol,n_stages)

    V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs=rise_time(delta_array,alpha_array,V_A_in_array,V_C_in_array,n_stages)
    
    V_A_max=np.max(V_A_measurment_abs)
    V_C_max=np.max(V_C_measurment_abs)
    
    if(abs(output_pol[0]-reference_point)<1e-4 and V_A_max<3 and V_C_max<3):
        cond=0
    
    print("V_A diff -> ",V_A_max)
    print("V_C diff -> ",V_C_max)
    print("Output pol", output_pol)
    print(abs(output_pol[0]-reference_point))
    print("gbest ->",gbest_obj)

V_A diff ->  2.217009424468735
V_C diff ->  1.933878092861498
Output pol [ 1.07950315e+00 -6.82209385e-07  1.66867940e-09]
2.156053113822054e-13
gbest -> 0.44101865110590055


In [10]:
print("The values of delta for the state -circ are:")
print(delta_array)
print("The values of delta for the state -circ are:")
print(alpha_array)
print("The values of V_A for the state -circ are:")
print(V_A_measurment_val)
print("The values of V_C for the state -circ are:")
print(V_C_measurment_val)
print("----------")
print("Output polarization state")
print("S1=%f, S2=%f, S3=%f"%(output_pol[0],output_pol[1],output_pol[2]))   

V_A_measurment_val_H=V_A_measurment_val
V_C_measurment_val_H=V_C_measurment_val

file1 = open("registo_alpha.txt", "w")
file2 = open("registo_delta.txt", "w")
file1.write("H\n")
file2.write("H\n")
str_alpha=repr(alpha_array)
file1.write(str_alpha+"\n")
str_delta=repr(delta_array)
file2.write(str_delta+"\n")
file1.close()
file2.close()

The values of delta for the state -circ are:
[0.03553893 0.03212831 0.01849859 0.02294987 0.02608799 0.03541481]
The values of delta for the state -circ are:
[4.57289489 4.44343606 4.4497941  4.36837577 4.4705977  4.34265005]
The values of V_A for the state -circ are:
[-12.33633719 -10.49447405  -9.19624858 -11.6281318   -8.40720797
  -8.67968817]
The values of V_C for the state -circ are:
[6.20726015 7.44935918 8.16688782 9.50341191 9.89192727 8.38694451]
----------
Output polarization state
S1=1.079503, S2=-0.000001, S3=0.000000


## H $\rightarrow$ V

In [45]:
output_pol=np.zeros(3)
V_A_max=100
V_C_max=100
cond=1
while(cond==1):
    
    ''' Important parameters '''
    goal_array=np.array([-1,0,0])
    ''' -------------------------- '''
    X_delta,X_alpha=particle_space(r_start_delta_array_A,r_start_delta_array_C,r_end_delta_array_A,r_end_delta_array_C,r_start_alpha_array_A,r_start_alpha_array_C,r_end_alpha_array_A,r_end_alpha_array_C,n_stages,n_particles)
    
    X = np.concatenate((X_delta, X_alpha))

    V = np.random.randn(n_stages*2, n_particles) * 0.1 #distribuição normal

    V_A_in_array=V_A_measurment_val_H
    V_C_in_array=V_C_measurment_val_H



    gbest_obj,gbest=PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,input_pol,150,3,10,10,0,n_stages)
    ''' 2 stages '''
    # gbest_obj,gbest=PSO(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,0,1,150,20,10,10,n_stages)

    # gbest_obj,gbest=PSO(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,0,1,150,15,5,5)

    output_pol=output_calc(gbest,input_pol,n_stages)
        
    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]
    
    V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs=rise_time(delta_array,alpha_array,V_A_in_array,V_C_in_array,n_stages)
        
    V_A_max=np.max(V_A_measurment_abs)
    V_C_max=np.max(V_C_measurment_abs)
    ''' 8 stages '''
    # if(abs(output_pol[0]+1)<1e-4 and V_A_max<7 and V_C_max<7):
    #     cond=0
    ''' 6 stages '''
    if(abs(output_pol[0]+reference_point)<1e-4 and V_A_max<7 and V_C_max<7):
        cond=0
    ''' 3 stages '''
    # if(abs(output_pol[0]+1)<1e-4 and V_A_max<13.5 and V_C_max<13.5):
    ''' 2 stages '''
    # if(abs(output_pol[0]+1)<1e-4 and V_A_max<20 and V_C_max<20):
        #cond=0  
    # if(abs(output_pol[0]+1)<1e-4):
        
    #     cond=0

    print("V_A diff -> ",V_A_max)
    print("V_C diff -> ",V_C_max)
    print("Output pol", output_pol)
    print("gbest ->",gbest_obj)

V_A diff ->  5.636831703526429
V_C diff ->  5.904656350371388
Output pol [-1.07950315e+00 -3.43253843e-11 -1.28050973e-12]
gbest -> 2.768077564269826


In [46]:
print("The values of delta for the state -circ are:")
print(delta_array)
print("The values of delta for the state -circ are:")
print(alpha_array)
print("The values of V_A for the state -circ are:")
print(V_A_measurment_val)
print("The values of V_C for the state -circ are:")
print(V_C_measurment_val)
print("----------")
print("Output polarization state")
print("S1=%f, S2=%f, S3=%f"%(output_pol[0],output_pol[1],output_pol[2]))   


file1 = open("registo_alpha.txt", "a")
file2 = open("registo_delta.txt", "a")
file1.write("V\n")
file2.write("V\n")
str_alpha=repr(alpha_array)
file1.write(str_alpha+"\n")
str_delta=repr(delta_array)
file2.write(str_delta+"\n")
file1.close()
file2.close()

V_A_measurment_val_V=V_A_measurment_val
V_C_measurment_val_V=V_C_measurment_val

The values of delta for the state -circ are:
[0.0271199  0.03620306 0.06778372 0.08227556 0.07678063 0.04912901]
The values of delta for the state -circ are:
[1.69913154 1.71313756 1.60363449 1.59613369 1.69826476 1.74512953]
The values of V_A for the state -circ are:
[-9.04139855 -7.07720138 -4.71724719 -6.35805951 -2.77037627 -4.4906885 ]
The values of V_C for the state -circ are:
[ 9.66777893 11.2475886  12.93394266 15.40806826 15.13448169 13.05490357]
----------
Output polarization state
S1=-1.079503, S2=-0.000000, S3=-0.000000


## H $\rightarrow$ +45

In [13]:
output_pol=np.zeros(3)
V_A_max=100
V_C_max=100
cond=1

V_A_in_array=np.concatenate((V_A_measurment_val_H, V_A_measurment_val_V))
V_C_in_array=np.concatenate((V_C_measurment_val_H, V_C_measurment_val_V))

V_A_in_array=np.reshape(V_A_in_array,(2,n_stages))
V_C_in_array=np.reshape(V_C_in_array,(2,n_stages))


while(cond==1):
    
    ''' important parameters '''
    goal_array=np.array([0,1,0])
    ''' -------------------------- '''
    
    X_delta,X_alpha=particle_space(r_start_delta_array_A,r_start_delta_array_C,r_end_delta_array_A,r_end_delta_array_C,r_start_alpha_array_A,r_start_alpha_array_C,r_end_alpha_array_A,r_end_alpha_array_C,n_stages,n_particles)

    X = np.concatenate((X_delta, X_alpha))

    V = np.random.randn(n_stages*2, n_particles) * 0.1 #distribuição normal
    
    
    gbest_obj,gbest=PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,input_pol,150,3,10,10,1,n_stages)
    ''' 2 stages '''
    # gbest_obj,gbest=PSO(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,0,1,150,25,10,10,n_stages)


    output_pol=output_calc(gbest,input_pol,n_stages)
    
    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]
    
    V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs=rise_time(delta_array,alpha_array,V_A_in_array[0],V_C_in_array[0],n_stages)
        
    V_A_max=np.max(V_A_measurment_abs)
    V_C_max=np.max(V_C_measurment_abs)
    ''' 8 stages '''
    # if(abs(output_pol[1]-1)<1e-4 and V_A_max<7 and V_C_max<7):
    #     cond=0
    ''' 6 stages '''
    if(abs(output_pol[1]-reference_point)<1e-4 and V_A_max<8.5 and V_C_max<8.5):
        cond=0
    ''' 3 stages '''
    #if(abs(output_pol[1]-1)<1e-4 and V_A_max<13.5 and V_C_max<13.5):
    ''' 2 stages '''
    # if(abs(output_pol[1]-1)<1e-4 and V_A_max<20 and V_C_max<20):

    #     cond=0
    # if(abs(output_pol[1]-1)<1e-4):
    #     cond=0
    
    print("V_A diff -> ",V_A_max)
    print("V_C diff -> ",V_C_max)
    print("Output pol", output_pol)
    print("gbest ->",gbest_obj)

V_A diff ->  5.171374195964707
V_C diff ->  9.748881667883609
Output pol [-5.41878263e-13  1.07950315e+00  1.99345001e-11]
gbest -> 3.5635645245469343
V_A diff ->  3.9289279414840284
V_C diff ->  8.787767923273526
Output pol [-6.31859657e-08  1.07950315e+00  4.33961218e-06]
gbest -> 3.314340303578529
V_A diff ->  3.974366818998295
V_C diff ->  16.33893574408344
Output pol [5.63678736e-04 1.07950300e+00 3.80879512e-06]
gbest -> 2.5792539550519242
V_A diff ->  3.288080908780767
V_C diff ->  12.242239767630288
Output pol [-0.01762464  1.07529824  0.09354206]
gbest -> 1.9910118340447405
V_A diff ->  1.7895655763184717
V_C diff ->  12.288442250302849
Output pol [-2.19675675e-08  1.07882642e+00  3.82178345e-02]
gbest -> 1.7158110597101837
V_A diff ->  1.3439017315840918
V_C diff ->  7.801117025810495
Output pol [-1.69975811e-07  1.07950315e+00 -4.47598228e-07]
gbest -> 1.153163006212083


In [14]:
print("The values of delta for the state -circ are:")
print(delta_array)
print("The values of delta for the state -circ are:")
print(alpha_array)
print("The values of V_A for the state -circ are:")
print(V_A_measurment_val)
print("The values of V_C for the state -circ are:")
print(V_C_measurment_val)
print("----------")
print("Output polarization state")
print("S1=%f, S2=%f, S3=%f"%(output_pol[0],output_pol[1],output_pol[2]))   

file1 = open("registo_alpha.txt", "a")
file2 = open("registo_delta.txt", "a")
file1.write("+45\n")
file2.write("+45\n")
str_alpha=repr(alpha_array)
file1.write(str_alpha+"\n")
str_delta=repr(delta_array)
file2.write(str_delta+"\n")
file1.close()
file2.close() 


The values of delta for the state -circ are:
[0.07271577 0.02441987 0.08767016 0.02696741 0.04726887 0.02653106]
The values of delta for the state -circ are:
[0.46836974 0.75443875 0.49861322 0.60182176 0.52683456 0.73136461]
The values of V_A for the state -circ are:
[-12.56725969  -9.3933139  -10.54015031 -11.32567887  -8.41831067
  -7.74182905]
The values of V_C for the state -circ are:
[13.83875434 11.49957888 15.96800485 13.16845826 15.16610845 12.96973451]
----------
Output polarization state
S1=-0.000000, S2=1.079503, S3=-0.000000


In [15]:
V_A_measurment_val_p45=V_A_measurment_val
V_C_measurment_val_p45=V_C_measurment_val

## H $\rightarrow$ -45

In [16]:
output_pol=np.zeros(3)
V_A_max=100
V_C_max=100
cond=1

V_A_in_array=np.concatenate((V_A_measurment_val_H, V_A_measurment_val_V))
V_C_in_array=np.concatenate((V_C_measurment_val_H, V_C_measurment_val_V))

V_A_in_array=np.concatenate((V_A_in_array, V_A_measurment_val_p45))
V_C_in_array=np.concatenate((V_C_in_array, V_C_measurment_val_p45))


V_A_in_array=np.reshape(V_A_in_array,(3,n_stages))
V_C_in_array=np.reshape(V_C_in_array,(3,n_stages))

while(cond==1):
    
    ''' important parameters '''
    goal_array=np.array([0,-1,0])
    ''' -------------------------- '''

    X_delta,X_alpha=particle_space(r_start_delta_array_A,r_start_delta_array_C,r_end_delta_array_A,r_end_delta_array_C,r_start_alpha_array_A,r_start_alpha_array_C,r_end_alpha_array_A,r_end_alpha_array_C,n_stages,n_particles)


    X = np.concatenate((X_delta, X_alpha))

    V = np.random.randn(n_stages*2, n_particles) * 0.1 #distribuição normal
    


    gbest_obj,gbest=PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,input_pol,150,2,10,10,1,n_stages)
    ''' 2 stages '''
    # gbest_obj,gbest=PSO(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,0,1,150,25,10,10,n_stages)


    output_pol=output_calc(gbest,input_pol,n_stages)
    
    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]
    
    V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs=rise_time(delta_array,alpha_array,V_A_in_array[0],V_C_in_array[0],n_stages)
        
    V_A_max=np.max(V_A_measurment_abs)
    V_C_max=np.max(V_C_measurment_abs)
    ''' 8 stages '''
    # if(abs(output_pol[1]+1)<1e-4 and V_A_max<7 and V_C_max<7):
    #     cond=0
    ''' 6 stages '''
    if(abs(output_pol[1]+reference_point)<1e-4 and V_A_max<6 and V_C_max<6):
        cond=0
    ''' 3 stages '''
    #if(abs(output_pol[1]+1)<1e-4 and V_A_max<13.5 and V_C_max<13.5):
    ''' 2 stages '''
    # if(abs(output_pol[1]+1)<1e-4 and V_A_max<20 and V_C_max<20):

    #     cond=0  
    # if(abs(output_pol[1]+1)<1e-4):
        
    #     cond=0
    

    print("V_A diff -> ",V_A_max)
    print("V_C diff -> ",V_C_max)
    print("Output pol", output_pol)
    print("gbest ->",gbest_obj)


V_A diff ->  7.082131957733514
V_C diff ->  1.7947175615231759
Output pol [ 3.55318932e-07 -1.07950315e+00 -8.29929458e-09]
gbest -> 0.8723972036418796
V_A diff ->  6.813557896703144
V_C diff ->  1.190358668650429
Output pol [-1.28279169e-08 -1.07950315e+00  1.34857648e-07]
gbest -> 0.8411295967813073
V_A diff ->  7.430371483853373
V_C diff ->  1.528337155533733
Output pol [-3.79640920e-10 -1.07950315e+00  1.23367170e-09]
gbest -> 0.8590881624686254
V_A diff ->  7.41598037529195
V_C diff ->  1.2809512021957712
Output pol [ 1.35344226e-10 -1.07950315e+00 -3.61875596e-09]
gbest -> 0.8339642975187045
V_A diff ->  8.103470117069604
V_C diff ->  2.0584157533147938
Output pol [-1.40609952e-14 -1.07950315e+00  5.23985079e-09]
gbest -> 0.9546154599952765
V_A diff ->  8.417487856903763
V_C diff ->  1.1856757809714802
Output pol [ 5.18280668e-12 -1.07950315e+00 -9.05402193e-14]
gbest -> 0.8874044285797374
V_A diff ->  6.956707594004174
V_C diff ->  0.8009955828872997
Output pol [ 1.09837797e-11 

In [17]:
print("The values of delta for the state -circ are:")
print(delta_array)
print("The values of delta for the state -circ are:")
print(alpha_array)
print("The values of V_A for the state -circ are:")
print(V_A_measurment_val)
print("The values of V_C for the state -circ are:")
print(V_C_measurment_val)
print("----------")
print("Output polarization state")
print("S1=%f, S2=%f, S3=%f"%(output_pol[0],output_pol[1],output_pol[2]))   

file1 = open("registo_alpha.txt", "a")
file2 = open("registo_delta.txt", "a")
file1.write("-45\n")
file2.write("-45\n")
str_alpha=repr(alpha_array)
file1.write(str_alpha+"\n")
str_delta=repr(delta_array)
file2.write(str_delta+"\n")
file1.close()
file2.close()

The values of delta for the state -circ are:
[0.04565401 0.0619087  0.04457535 0.06202406 0.03888721 0.05312143]
The values of delta for the state -circ are:
[2.71854771 2.66522345 2.69171689 2.71201763 2.83929723 2.71448541]
The values of V_A for the state -circ are:
[-7.33666565 -4.68611249 -5.20944241 -6.34682726 -4.69681167 -3.69536026]
The values of V_C for the state -circ are:
[ 7.07587617  8.0520797   8.20288804  9.3263569  10.13788765  9.38950463]
----------
Output polarization state
S1=-0.000000, S2=-1.079503, S3=0.000000


In [18]:
V_A_measurment_val_n45=V_A_measurment_val
V_C_measurment_val_n45=V_C_measurment_val

## H $\rightarrow$ $\curvearrowright$

In [37]:
output_pol=np.zeros(3)
V_A_max=100
V_C_max=100
cond=1

V_A_in_array=np.concatenate((V_A_measurment_val_H, V_A_measurment_val_V))
V_C_in_array=np.concatenate((V_C_measurment_val_H, V_C_measurment_val_V))

V_A_in_array=np.concatenate((V_A_in_array, V_A_measurment_val_p45))
V_C_in_array=np.concatenate((V_C_in_array, V_C_measurment_val_p45))

V_A_in_array=np.concatenate((V_A_in_array, V_A_measurment_val_n45))
V_C_in_array=np.concatenate((V_C_in_array, V_C_measurment_val_n45))


V_A_in_array=np.reshape(V_A_in_array,(4,n_stages))
V_C_in_array=np.reshape(V_C_in_array,(4,n_stages))


while(cond==1):
    
    ''' important parameters '''
    goal_array=np.array([0,0,1])
    ''' -------------------------- '''

    X_delta,X_alpha=particle_space(r_start_delta_array_A,r_start_delta_array_C,r_end_delta_array_A,r_end_delta_array_C,r_start_alpha_array_A,r_start_alpha_array_C,r_end_alpha_array_A,r_end_alpha_array_C,n_stages,n_particles)


    X = np.concatenate((X_delta, X_alpha))

    V = np.random.randn(n_stages*2, n_particles) * 0.1 #distribuição normal
    

    
    gbest_obj,gbest=PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,input_pol,150,2,10,10,1,n_stages)
    ''' 2 stages '''
    # gbest_obj,gbest=PSO(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,0,1,150,25,10,10,n_stages)


    output_pol=output_calc(gbest,input_pol,n_stages)
    
    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]
    
    V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs=rise_time(delta_array,alpha_array,V_A_in_array[0],V_C_in_array[0],n_stages)
    
    
        
    V_A_max=np.max(V_A_measurment_abs)
    V_C_max=np.max(V_C_measurment_abs)

    V_A_measurment_val_ex,V_C_measurment_val_ex,V_A_measurment_abs_ex,V_C_measurment_abs_ex=rise_time(delta_array,alpha_array,V_A_measurment_val_p45,V_C_measurment_val_p45,n_stages)
    
    V_A_max_ex=np.max(V_A_measurment_abs_ex)
    V_C_max_ex=np.max(V_C_measurment_abs_ex)
    # if(abs(output_pol[2]-1)<1e-4 and V_A_max<7 and V_C_max<7):
    #     cond=0
    if(abs(output_pol[2]-reference_point)<1e-4 and V_A_max<8.06 and V_C_max<8.06):
        cond=0
    #if(abs(output_pol[2]-1)<1e-4 and V_A_max<13.5 and V_C_max<13.5):
    # if(abs(output_pol[2]-1)<1e-4 and V_A_max<20 and V_C_max<20):
    
   
    #     cond=0  
    # if(abs(output_pol[2]-1)<1e-4):
    #     cond=0
    #print(V_A_max_ex)
    #print(V_C_max_ex)
    print("V_A diff -> ",V_A_max)
    print("V_C diff -> ",V_C_max)
    print("Output pol", output_pol)
    print("gbest ->",gbest_obj)

V_A diff ->  9.986485520317038
V_C diff ->  8.170318625333177
Output pol [-9.36507210e-10  6.34080134e-12  1.07950315e+00]
gbest -> 0.96782121547316
V_A diff ->  7.880138246638725
V_C diff ->  8.045667396345204
Output pol [-1.51480448e-08 -1.38853527e-09  1.07950315e+00]
gbest -> 0.9286447434531078


In [40]:


print("The values of delta for the state -circ are:")
print(delta_array)
print("The values of delta for the state -circ are:")
print(alpha_array)
print("The values of V_A for the state -circ are:")
print(V_A_measurment_val)
print("The values of V_C for the state -circ are:")
print(V_C_measurment_val)
print("----------")
print("Output polarization state")
print("S1=%f, S2=%f, S3=%f"%(output_pol[0],output_pol[1],output_pol[2]))   

file1 = open("registo_alpha.txt", "a")
file2 = open("registo_delta.txt", "a")
file1.write("circ+\n")
file2.write("circ+\n")
str_alpha=repr(alpha_array)
file1.write(str_alpha+"\n")
str_delta=repr(delta_array)
file2.write(str_delta+"\n")
file1.close()
file2.close()

The values of delta for the state -circ are:
[0.10271409 0.09374465 0.10608228 0.07574369 0.10519349 0.10744969]
The values of delta for the state -circ are:
[1.69159823 1.79984141 1.59613623 1.81840365 1.74014891 1.62152802]
The values of V_A for the state -circ are:
[-4.45619894 -3.17806324 -2.6231463  -5.90785045 -0.84765194 -1.45712623]
The values of V_C for the state -circ are:
[13.25004946 13.33807245 14.97635467 14.00930921 16.1630696  16.43261191]
----------
Output polarization state
S1=-0.000000, S2=-0.000000, S3=1.079503


In [41]:
V_A_measurment_val_pcirc=V_A_measurment_val
V_C_measurment_val_pcirc=V_C_measurment_val

## H $\rightarrow$ $\curvearrowleft$


In [22]:
output_pol=np.zeros(3)
V_A_max=100
V_C_max=100
cond=1

V_A_in_array=np.concatenate((V_A_measurment_val_H, V_A_measurment_val_V))
V_C_in_array=np.concatenate((V_C_measurment_val_H, V_C_measurment_val_V))

V_A_in_array=np.concatenate((V_A_in_array, V_A_measurment_val_p45))
V_C_in_array=np.concatenate((V_C_in_array, V_C_measurment_val_p45))

V_A_in_array=np.concatenate((V_A_in_array, V_A_measurment_val_n45))
V_C_in_array=np.concatenate((V_C_in_array, V_C_measurment_val_n45))

V_A_in_array=np.concatenate((V_A_in_array, V_A_measurment_val_pcirc))
V_C_in_array=np.concatenate((V_C_in_array, V_C_measurment_val_pcirc))


V_A_in_array=np.reshape(V_A_in_array,(5,n_stages))
V_C_in_array=np.reshape(V_C_in_array,(5,n_stages))

while(cond==1):
    
    ''' important parameters '''
    goal_array=np.array([0,0,-1])
    ''' -------------------------- '''
    
    X_delta,X_alpha=particle_space(r_start_delta_array_A,r_start_delta_array_C,r_end_delta_array_A,r_end_delta_array_C,r_start_alpha_array_A,r_start_alpha_array_C,r_end_alpha_array_A,r_end_alpha_array_C,n_stages,n_particles)


    X = np.concatenate((X_delta, X_alpha))

    V = np.random.randn(n_stages*2, n_particles) * 0.1 #distribuição normal
    
    
    gbest_obj,gbest=PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,input_pol,150,2,10,10,1,n_stages)
    ''' 2 stages '''
    #gbest_obj,gbest=PSO(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,0,1,150,25,10,10,n_stages)


    output_pol=output_calc(gbest,input_pol,n_stages)
    
    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]
    
    V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs=rise_time(delta_array,alpha_array,V_A_in_array[0],V_C_in_array[0],n_stages)
    # V_A_measurment_val_ex,V_C_measurment_val_ex,V_A_measurment_abs_ex,V_C_measurment_abs_ex=rise_time(delta_array,alpha_array,V_A_measurment_val_n45,V_C_measurment_val_n45,n_stages)

    V_A_max=np.max(V_A_measurment_abs)
    V_C_max=np.max(V_C_measurment_abs)
    
    # V_A_max_ex=np.max(V_A_measurment_abs_ex)
    # V_C_max_ex=np.max(V_C_measurment_abs_ex)
    
    # n45 para circular negativa
    ''' 8 stages '''
    # if(abs(output_pol[2]+1)<1e-4 and V_A_max<7.8 and V_C_max<7.8 and V_A_max_ex<7.8 and V_C_max_ex<7.8):
    #     cond=0
    # if(abs(output_pol[2]+1)<1e-4 and V_A_max<7 and V_C_max<7):
    #     cond=0
    ''' 6 stages '''
    # if(abs(output_pol[2]+reference_point)<1e-4 and V_A_max<8.8 and V_C_max<8.8):
    #     cond=0
    if(abs(output_pol[2]+reference_point)<1e-4 and V_A_max<5 and V_C_max<5 ):
        cond=0
    ''' 3 stages '''
    #if(abs(output_pol[2]+1)<1e-4 and V_A_max<13.5 and V_C_max<13.5 and V_A_max_ex<13.5 and V_C_max_ex<13.5):
    ''' 2 stages '''
    # if(abs(output_pol[2]+1)<1e-4 and V_A_max<20 and V_C_max<20 and V_A_max_ex<20 and V_C_max_ex<20):

    #     cond=0
    # if(abs(output_pol[2]+1)<1e-4):
        
    #     cond=0
    # print(V_A_max_ex)
    # print(V_C_max_ex)    
    print("V_A diff -> ",V_A_max)
    print("V_C diff -> ",V_C_max)
    print("Output pol", output_pol)
    print("gbest ->",gbest_obj)

V_A diff ->  5.457539337790931
V_C diff ->  4.860615021301132
Output pol [ 8.23867925e-09 -1.16664824e-06 -1.07950315e+00]
gbest -> 0.4631741553752556
V_A diff ->  4.836078664918034
V_C diff ->  4.010929073354045
Output pol [-1.04984313e-12  6.82458815e-12 -1.07950315e+00]
gbest -> 0.450741619474477


In [23]:

print("The values of delta for the state -circ are:")
print(delta_array)
print("The values of delta for the state -circ are:")
print(alpha_array)
print("The values of V_A for the state -circ are:")
print(V_A_measurment_val)
print("The values of V_C for the state -circ are:")
print(V_C_measurment_val)
print("----------")
print("Output polarization state")
print("S1=%f, S2=%f, S3=%f"%(output_pol[0],output_pol[1],output_pol[2]))   

file1 = open("registo_alpha.txt", "a")
file2 = open("registo_delta.txt", "a")
file1.write("circ-\n")
file2.write("circ-\n")
str_alpha=repr(alpha_array)
file1.write(str_alpha+"\n")
str_delta=repr(delta_array)
file2.write(str_delta+"\n")
file1.close()
file2.close()

The values of delta for the state -circ are:
[ 0.00702162  0.03848801  0.00081359  0.00945274  0.05780356 -0.02428879]
The values of delta for the state -circ are:
[1.80681051 1.66760677 1.74954598 1.77987312 1.83125736 1.71214007]
The values of V_A for the state -circ are:
[-10.23617672  -7.02304666  -8.44867968 -10.2924506   -3.57112931
  -9.09974571]
The values of V_C for the state -circ are:
[ 8.67895    11.46028825  9.43514774 11.48741521 13.7586701   9.78347892]
----------
Output polarization state
S1=-0.000000, S2=0.000000, S3=-1.079503


In [24]:
V_A_measurment_val_ncirc=V_A_measurment_val
V_C_measurment_val_ncirc=V_C_measurment_val

## V $\rightarrow$ +45

In [50]:
V_A_Calc=V_A_measurment_val_V-V_A_measurment_val_p45
V_C_Calc=V_C_measurment_val_V-V_C_measurment_val_p45

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)


print("TRansition results:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Max voltage interval",np.max(V_A_Calc))

else:
    print("Max voltage interval",np.max(V_C_Calc))

TRansition results:
Max voltage interval 5.82290311441572


## V $\rightarrow$ -45

In [49]:
V_A_Calc=V_A_measurment_val_V-V_A_measurment_val_n45
V_C_Calc=V_C_measurment_val_V-V_C_measurment_val_n45

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("Transition results:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Max voltage interval",np.max(V_A_Calc))

else:
    print("Max voltage interva",((np.max(V_C_Calc)*1000)/5876),"ns",np.max(V_C_Calc))

Transition results:
Max voltage interva 1.0350087403525998 ns 6.0817113583118765


## V $\rightarrow$ circ+

In [48]:
V_A_Calc=V_A_measurment_val_V-V_A_measurment_val_pcirc
V_C_Calc=V_C_measurment_val_V-V_C_measurment_val_pcirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("Transition results:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Max voltage interval",np.max(V_A_Calc))

else:
    print("Max voltage interval",np.max(V_C_Calc))

Transition results:
Max voltage interval 4.585199607484955


## V $\rightarrow$ circ-

In [47]:
V_A_Calc=V_A_measurment_val_V-V_A_measurment_val_ncirc
V_C_Calc=V_C_measurment_val_V-V_C_measurment_val_ncirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("Transition results:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Max voltage interval",np.max(V_A_Calc))

else:
    print("Max voltage interval",np.max(V_C_Calc))

Transition results:
Max voltage interval 4.60905721190807


## +45  $\rightarrow$ -45

In [29]:
V_A_Calc=V_A_measurment_val_p45-V_A_measurment_val_n45
V_C_Calc=V_C_measurment_val_p45-V_C_measurment_val_n45

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("Transition results:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Max voltage interval",np.max(V_A_Calc))

else:
    print("Max voltage interval",np.max(V_C_Calc))

Transition results:
Max voltage interval 7.765116811416753


## +45 $\rightarrow$ circ+

In [42]:
V_A_Calc=V_A_measurment_val_p45-V_A_measurment_val_pcirc
V_C_Calc=V_C_measurment_val_p45-V_C_measurment_val_pcirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("Transition results:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Max voltage interval",np.max(V_A_Calc))

else:
    print("Max voltage interval",np.max(V_C_Calc))

Transition results:
Max voltage interval 8.111060747999094


## +45 $\rightarrow$ circ-

In [31]:
V_A_Calc=V_A_measurment_val_p45-V_A_measurment_val_ncirc
V_C_Calc=V_C_measurment_val_p45-V_C_measurment_val_ncirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("Transition results:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Max voltage interval",np.max(V_A_Calc))

else:
    print("Max voltage interval",np.max(V_C_Calc))

Transition results:
Max voltage interval 6.532857106583672


## -45 $\rightarrow$ circ+

In [44]:
V_A_Calc=V_A_measurment_val_n45-V_A_measurment_val_pcirc
V_C_Calc=V_C_measurment_val_n45-V_C_measurment_val_pcirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("Transition results:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Max voltage interval",np.max(V_A_Calc))

else:
    print("Max voltage interval",np.max(V_C_Calc))

Transition results:
Max voltage interval 7.043107279023278


## -45 $\rightarrow$ circ-

In [33]:
V_A_Calc=V_A_measurment_val_n45-V_A_measurment_val_ncirc
V_C_Calc=V_C_measurment_val_n45-V_C_measurment_val_ncirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("Transition results:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Max voltage interval",np.max(V_A_Calc))

else:
    print("Max voltage interval",np.max(V_C_Calc))

Transition results:
Max voltage interval 5.404385453779486


## circ+ $\rightarrow$ circ-

In [43]:
V_A_Calc=V_A_measurment_val_pcirc-V_A_measurment_val_ncirc
V_C_Calc=V_C_measurment_val_pcirc-V_C_measurment_val_ncirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("Transition results:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Max voltage interval",np.max(V_A_Calc))

else:
    print("Max voltage interval",np.max(V_C_Calc))
    

Transition results:
Max voltage interval 7.64261948168911


In [35]:
end = time.time()
print(end - start)

127.95106482505798


In [36]:
''' Store the seed if it's good '''
# seed_4stages6_t_7_76=Seed
# %store seed_4stages6_t_7_76